In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar100.load_data()
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0
y_train = y_train[:,0].astype(np.int32)
y_test = y_test[:,0].astype(np.int32)

#Creating validation sets
val_size = 2000
X_valid, X_train = X_train[:val_size], X_train[val_size:]
y_valid, y_train = y_train[:val_size], y_train[val_size:]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_hidden1 = 4096
n_hidden2 = 4096
n_outputs = len(np.unique(y_train))

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z

############

conv1_fmaps = 96
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 128
conv2_ksize = 5
conv2_stride = 1
conv2_pad = "SAME"

conv3_fmaps = 256
conv3_ksize = 5
conv3_stride = 1
conv3_pad = "SAME"

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, 32, 32, 3], name="X")
    y = tf.placeholder(tf.int64, shape=(None), name="y")

conv1 = tf.layers.conv2d(X, filters=conv1_fmaps, kernel_size=conv1_ksize, 
                         strides=conv1_stride, padding=conv1_pad, name="conv1")  
bn1 = tf.layers.batch_normalization(conv1, training=True, momentum=0.9)
bn1_act = tf.nn.relu(bn1)

pool1 = tf.nn.max_pool(bn1_act, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")

conv2 = tf.layers.conv2d(pool1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad, name="conv2")   
bn2 = tf.layers.batch_normalization(conv2, training=True, momentum=0.9)
bn2_act = tf.nn.relu(bn2)

pool2 = tf.nn.max_pool(bn2_act, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")

conv3 = tf.layers.conv2d(pool2, filters=conv3_fmaps, kernel_size=conv3_ksize,
                         strides=conv3_stride, padding=conv3_pad, name="conv3")
bn3 = tf.layers.batch_normalization(conv3, training=True, momentum=0.9)
bn3_act = tf.nn.relu(bn3)

pool3 = tf.nn.max_pool(bn3_act, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME")

n_fc1 = np.int64(pool3.shape[1]*pool3.shape[2]*pool3.shape[3])
pool3_flat = tf.reshape(pool3, shape=[-1, n_fc1])

###################

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(pool3_flat, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")

def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        a = 3.7
        lambdaVal = tf.constant(lambdaValScalar, W.dtype)
        aLambda = tf.scalar_mul(a, lambdaVal)
        condMat = tf.multiply(tf.cast(tf.abs(W) > lambdaVal, W.dtype), tf.cast(tf.abs(W) <= aLambda, W.dtype))
        lambdaValSq = tf.pow(lambdaVal, 2)
        absW = tf.abs(W)
        reg_value = tf.cast(absW <= lambdaVal, W.dtype)*tf.scalar_mul(lambdaVal, absW) + \
                    condMat*tf.scalar_mul(-0.5/(a-1), tf.pow(W, 2) - 2*tf.scalar_mul(aLambda, absW) + lambdaValSq) + \
                    tf.cast(absW > aLambda, W.dtype)*tf.scalar_mul(0.5*(a+1), lambdaValSq)
        return tf.reduce_sum(reg_value)
    elif type == 9:
        b = 2
        lambdaVal = tf.constant(lambdaValScalar, W.dtype)
        absW = tf.abs(W)
        reg_value = tf.cast(absW > b*lambdaVal, W.dtype)*tf.sign(W)*(0.5*b*tf.pow(lambdaVal, 2)) + \
                    tf.cast(absW <= b*lambdaVal, W.dtype)*tf.sign(W)*tf.scalar_mul(lambdaVal, absW - \
                                                                       tf.scalar_mul(1/(2*b*lambdaVal), tf.pow(W, 2))) 
        return tf.reduce_sum(reg_value)
    elif type == 10:
        gammaVal = pow(10, 0)
        #gammaVal = tf.sqrt(tf.abs(tf.div(tf.reduce_sum(W), tf.reduce_sum(tf.pow(W, 3)))))
        reg_value =  (2/math.pi)*tf.atan(tf.scalar_mul(gammaVal, tf.abs(W)))
        return tf.reduce_sum(reg_value)
    elif type == 11:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 10.0*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 11
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
                                  
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    

    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices,:,:,:], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, acc_val = sess.run([loss_total, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        
        print("{}\tValidation losses: {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20200208153222.ckpt") # or better, use save_path
    n_batches = len(X_test) // 100
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        #print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [ ]:
   for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
     for i in [1, 2, 3]:
            print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
            execute_code(rand_seed=i, lambda_exp=j)


RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20210605132503
0	Validation losses: 18.1856	Best Loss: 18.1856 (0)	Accuracy: 26.80%
1	Validation losses: 15.0378	Best Loss: 15.0378 (0)	Accuracy: 32.70%
2	Validation losses: 11.5497	Best Loss: 11.5497 (0)	Accuracy: 37.25%
3	Validation losses: 8.4678	Best Loss: 8.4678 (0)	Accuracy: 39.10%
4	Validation losses: 5.8940	Best Loss: 5.8940 (0)	Accuracy: 44.80%
5	Validation losses: 4.3148	Best Loss: 4.3148 (0)	Accuracy: 46.15%
6	Validation losses: 3.6092	Best Loss: 3.6092 (0)	Accuracy: 44.00%
7	Validation losses: 2.9605	Best Loss: 2.9605 (0)	Accuracy: 50.30%
8	Validation losses: 2.8779	Best Loss: 2.8779 (0)	Accuracy: 48.45%
9	Validation losses: 2.8058	Best Loss: 2.8058 (0)	Accuracy: 49.10%
10	Validation losses: 2.8731	Best Loss: 2.8058 (1)	Accuracy: 48.25%
11	Validation losses: 2.9264	Best Loss: 2.8058 (2)	Accuracy: 47.45%
12	Validation losses: 2.9676	Best Loss: 2.8058 (3)	Accuracy: 48.75%
13	Validation losses: 3.0143	Best Loss: 2.8058 (4)	Accuracy: 46

20	Validation losses: 3.9684	Best Loss: 3.0137 (10)	Accuracy: 44.50%
21	Validation losses: 3.7392	Best Loss: 3.0137 (11)	Accuracy: 49.95%
22	Validation losses: 3.8409	Best Loss: 3.0137 (12)	Accuracy: 48.30%
23	Validation losses: 3.6979	Best Loss: 3.0137 (13)	Accuracy: 49.70%
24	Validation losses: 3.6345	Best Loss: 3.0137 (14)	Accuracy: 49.65%
25	Validation losses: 3.6978	Best Loss: 3.0137 (15)	Accuracy: 48.60%
26	Validation losses: 3.5326	Best Loss: 3.0137 (16)	Accuracy: 49.40%
27	Validation losses: 3.3974	Best Loss: 3.0137 (17)	Accuracy: 49.95%
28	Validation losses: 3.4006	Best Loss: 3.0137 (18)	Accuracy: 50.15%
29	Validation losses: 3.2242	Best Loss: 3.0137 (19)	Accuracy: 50.70%
30	Validation losses: 3.0669	Best Loss: 3.0137 (20)	Accuracy: 51.80%
31	Validation losses: 2.9951	Best Loss: 2.9951 (0)	Accuracy: 52.55%
32	Validation losses: 2.8997	Best Loss: 2.8997 (0)	Accuracy: 53.10%
33	Validation losses: 2.9669	Best Loss: 2.8997 (1)	Accuracy: 51.75%
34	Validation losses: 2.5945	Best Los

6	Validation losses: 4.5855	Best Loss: 4.5855 (0)	Accuracy: 46.20%
7	Validation losses: 3.9693	Best Loss: 3.9693 (0)	Accuracy: 50.00%
8	Validation losses: 3.7095	Best Loss: 3.7095 (0)	Accuracy: 48.55%
9	Validation losses: 3.4648	Best Loss: 3.4648 (0)	Accuracy: 49.40%
10	Validation losses: 3.3798	Best Loss: 3.3798 (0)	Accuracy: 49.20%
11	Validation losses: 3.4276	Best Loss: 3.3798 (1)	Accuracy: 48.65%
12	Validation losses: 3.3815	Best Loss: 3.3798 (2)	Accuracy: 49.55%
13	Validation losses: 3.4109	Best Loss: 3.3798 (3)	Accuracy: 50.25%
14	Validation losses: 3.6422	Best Loss: 3.3798 (4)	Accuracy: 48.50%
15	Validation losses: 3.6399	Best Loss: 3.3798 (5)	Accuracy: 48.20%
16	Validation losses: 3.7229	Best Loss: 3.3798 (6)	Accuracy: 48.25%
17	Validation losses: 3.9560	Best Loss: 3.3798 (7)	Accuracy: 46.90%
18	Validation losses: 3.8725	Best Loss: 3.3798 (8)	Accuracy: 48.05%
19	Validation losses: 3.9464	Best Loss: 3.3798 (9)	Accuracy: 47.85%
20	Validation losses: 4.1502	Best Loss: 3.3798 (10)	

59	Validation losses: 3.6433	Best Loss: 2.0882 (16)	Accuracy: 49.55%
60	Validation losses: 3.5140	Best Loss: 2.0882 (17)	Accuracy: 50.15%
61	Validation losses: 3.5754	Best Loss: 2.0882 (18)	Accuracy: 48.75%
62	Validation losses: 3.3590	Best Loss: 2.0882 (19)	Accuracy: 52.35%
63	Validation losses: 3.3068	Best Loss: 2.0882 (20)	Accuracy: 52.10%
Early stopping!
Total running time:	 38426
INFO:tensorflow:Restoring parameters from ./models/model_20210611225348.ckpt
Test error rate:    39.5699999630%
RANDOM SEED:   3 	LAMBDA_EXP:   -4.4
Now: 20210613093456
0	Validation losses: 9.4332	Best Loss: 9.4332 (0)	Accuracy: 24.70%
1	Validation losses: 8.4425	Best Loss: 8.4425 (0)	Accuracy: 36.60%
2	Validation losses: 7.6776	Best Loss: 7.6776 (0)	Accuracy: 36.70%
3	Validation losses: 6.6815	Best Loss: 6.6815 (0)	Accuracy: 44.45%
4	Validation losses: 5.9085	Best Loss: 5.9085 (0)	Accuracy: 44.00%
5	Validation losses: 5.1053	Best Loss: 5.1053 (0)	Accuracy: 47.50%
6	Validation losses: 4.4025	Best Loss: 4.

44	Validation losses: 2.0114	Best Loss: 2.0114 (0)	Accuracy: 60.10%
45	Validation losses: 2.0244	Best Loss: 2.0114 (1)	Accuracy: 59.45%
46	Validation losses: 2.0083	Best Loss: 2.0083 (0)	Accuracy: 58.95%
47	Validation losses: 2.0144	Best Loss: 2.0083 (1)	Accuracy: 59.40%
48	Validation losses: 2.2127	Best Loss: 2.0083 (2)	Accuracy: 54.45%
49	Validation losses: 2.5252	Best Loss: 2.0083 (3)	Accuracy: 54.85%
50	Validation losses: 2.5167	Best Loss: 2.0083 (4)	Accuracy: 54.40%
51	Validation losses: 2.4819	Best Loss: 2.0083 (5)	Accuracy: 55.90%
52	Validation losses: 2.4840	Best Loss: 2.0083 (6)	Accuracy: 55.30%
53	Validation losses: 2.6551	Best Loss: 2.0083 (7)	Accuracy: 54.35%
54	Validation losses: 2.6996	Best Loss: 2.0083 (8)	Accuracy: 53.10%
55	Validation losses: 2.7575	Best Loss: 2.0083 (9)	Accuracy: 53.25%
56	Validation losses: 2.7661	Best Loss: 2.0083 (10)	Accuracy: 54.05%
57	Validation losses: 2.8786	Best Loss: 2.0083 (11)	Accuracy: 52.45%
58	Validation losses: 3.1278	Best Loss: 2.0083

...

57	Validation losses: 2.7868	Best Loss: 2.0137 (9)	Accuracy: 54.10%
58	Validation losses: 2.8693	Best Loss: 2.0137 (10)	Accuracy: 53.35%
59	Validation losses: 2.9831	Best Loss: 2.0137 (11)	Accuracy: 52.05%
60	Validation losses: 3.1436	Best Loss: 2.0137 (12)	Accuracy: 51.75%
61	Validation losses: 2.8653	Best Loss: 2.0137 (13)	Accuracy: 54.40%
62	Validation losses: 2.8426	Best Loss: 2.0137 (14)	Accuracy: 55.75%
63	Validation losses: 2.7058	Best Loss: 2.0137 (15)	Accuracy: 56.90%
64	Validation losses: 2.5411	Best Loss: 2.0137 (16)	Accuracy: 57.60%
65	Validation losses: 2.4305	Best Loss: 2.0137 (17)	Accuracy: 58.05%
66	Validation losses: 2.3447	Best Loss: 2.0137 (18)	Accuracy: 59.05%
67	Validation losses: 2.2685	Best Loss: 2.0137 (19)	Accuracy: 59.40%
68	Validation losses: 2.2266	Best Loss: 2.0137 (20)	Accuracy: 59.00%
Early stopping!
Total running time:	 47882
INFO:tensorflow:Restoring parameters from ./models/model_20210622103805.ckpt
Test error rate:    40.0999997556%
RANDOM SEED:   1 	

31	Validation losses: 2.6772	Best Loss: 2.6772 (0)	Accuracy: 56.70%
32	Validation losses: 2.6075	Best Loss: 2.6075 (0)	Accuracy: 56.90%
33	Validation losses: 2.5300	Best Loss: 2.5300 (0)	Accuracy: 57.40%
34	Validation losses: 2.5137	Best Loss: 2.5137 (0)	Accuracy: 57.20%
35	Validation losses: 2.5048	Best Loss: 2.5048 (0)	Accuracy: 57.40%
36	Validation losses: 2.4874	Best Loss: 2.4874 (0)	Accuracy: 57.75%
37	Validation losses: 2.4731	Best Loss: 2.4731 (0)	Accuracy: 57.60%
38	Validation losses: 2.4746	Best Loss: 2.4731 (1)	Accuracy: 57.60%
39	Validation losses: 2.4671	Best Loss: 2.4671 (0)	Accuracy: 57.60%
40	Validation losses: 2.4664	Best Loss: 2.4664 (0)	Accuracy: 57.70%
41	Validation losses: 2.4632	Best Loss: 2.4632 (0)	Accuracy: 57.75%
42	Validation losses: 2.4601	Best Loss: 2.4601 (0)	Accuracy: 57.95%
43	Validation losses: 2.4416	Best Loss: 2.4416 (0)	Accuracy: 57.90%
44	Validation losses: 2.4063	Best Loss: 2.4063 (0)	Accuracy: 57.85%
45	Validation losses: 2.3932	Best Loss: 2.3932 (

1	Validation losses: 3.6842	Best Loss: 3.6842 (0)	Accuracy: 32.05%
2	Validation losses: 3.3918	Best Loss: 3.3918 (0)	Accuracy: 39.30%
3	Validation losses: 3.3014	Best Loss: 3.3014 (0)	Accuracy: 41.20%
4	Validation losses: 3.0825	Best Loss: 3.0825 (0)	Accuracy: 44.80%
5	Validation losses: 2.9311	Best Loss: 2.9311 (0)	Accuracy: 48.95%
6	Validation losses: 3.0328	Best Loss: 2.9311 (1)	Accuracy: 48.70%
7	Validation losses: 3.0575	Best Loss: 2.9311 (2)	Accuracy: 49.55%
8	Validation losses: 3.2341	Best Loss: 2.9311 (3)	Accuracy: 46.85%
9	Validation losses: 3.2999	Best Loss: 2.9311 (4)	Accuracy: 49.00%
10	Validation losses: 3.4603	Best Loss: 2.9311 (5)	Accuracy: 49.10%
11	Validation losses: 3.5091	Best Loss: 2.9311 (6)	Accuracy: 50.20%
12	Validation losses: 3.6745	Best Loss: 2.9311 (7)	Accuracy: 49.05%
13	Validation losses: 3.7706	Best Loss: 2.9311 (8)	Accuracy: 49.20%
14	Validation losses: 3.7187	Best Loss: 2.9311 (9)	Accuracy: 49.70%
15	Validation losses: 3.8587	Best Loss: 2.9311 (10)	Accur

3	Validation losses: 2.8740	Best Loss: 2.8740 (0)	Accuracy: 42.25%
4	Validation losses: 2.7382	Best Loss: 2.7382 (0)	Accuracy: 45.65%
5	Validation losses: 2.7094	Best Loss: 2.7094 (0)	Accuracy: 46.20%
6	Validation losses: 2.7648	Best Loss: 2.7094 (1)	Accuracy: 47.75%
7	Validation losses: 2.7987	Best Loss: 2.7094 (2)	Accuracy: 49.10%
8	Validation losses: 2.9357	Best Loss: 2.7094 (3)	Accuracy: 47.25%
9	Validation losses: 3.1182	Best Loss: 2.7094 (4)	Accuracy: 49.10%
10	Validation losses: 3.2540	Best Loss: 2.7094 (5)	Accuracy: 47.55%
11	Validation losses: 3.4385	Best Loss: 2.7094 (6)	Accuracy: 47.40%
12	Validation losses: 3.4317	Best Loss: 2.7094 (7)	Accuracy: 46.25%
13	Validation losses: 3.5669	Best Loss: 2.7094 (8)	Accuracy: 45.90%
14	Validation losses: 3.6840	Best Loss: 2.7094 (9)	Accuracy: 46.80%
15	Validation losses: 3.6509	Best Loss: 2.7094 (10)	Accuracy: 47.40%
16	Validation losses: 3.8571	Best Loss: 2.7094 (11)	Accuracy: 46.20%
17	Validation losses: 4.0283	Best Loss: 2.7094 (12)	A

8	Validation losses: 2.6748	Best Loss: 2.4966 (3)	Accuracy: 49.90%
9	Validation losses: 2.7642	Best Loss: 2.4966 (4)	Accuracy: 49.60%
10	Validation losses: 3.0000	Best Loss: 2.4966 (5)	Accuracy: 47.15%
11	Validation losses: 3.3242	Best Loss: 2.4966 (6)	Accuracy: 45.90%
12	Validation losses: 3.3734	Best Loss: 2.4966 (7)	Accuracy: 49.00%
13	Validation losses: 3.4129	Best Loss: 2.4966 (8)	Accuracy: 47.05%
14	Validation losses: 3.3656	Best Loss: 2.4966 (9)	Accuracy: 48.50%
15	Validation losses: 3.4772	Best Loss: 2.4966 (10)	Accuracy: 48.20%
16	Validation losses: 3.8828	Best Loss: 2.4966 (11)	Accuracy: 45.75%
17	Validation losses: 3.8338	Best Loss: 2.4966 (12)	Accuracy: 46.70%
18	Validation losses: 3.9541	Best Loss: 2.4966 (13)	Accuracy: 45.45%
19	Validation losses: 3.9723	Best Loss: 2.4966 (14)	Accuracy: 45.10%
20	Validation losses: 4.1155	Best Loss: 2.4966 (15)	Accuracy: 46.45%
21	Validation losses: 4.2134	Best Loss: 2.4966 (16)	Accuracy: 44.50%
22	Validation losses: 4.1135	Best Loss: 2.4